In [52]:
import os
import fiona
import pandas as pd
from pyproj import Transformer
from shapely.geometry import shape
import math
import numpy as np

In [45]:
def find_shapefiles(directory):
    
    shapefiles = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.shp'):
                shapefiles.append(os.path.join(root, file))
    return shapefiles

In [46]:
main_directory = '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024'  
shapefiles = find_shapefiles(main_directory)
print("Shapefiles found:", shapefiles)

Shapefiles found: ['//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\10Aug\\pe02_v1_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\11Aug\\pe02_h1e_hd_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\11Aug\\pe02_h1w_hd_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\11Aug\\pe02_h1_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\12Aug\\pe02_camp1_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\12Aug\\pe02_camp2_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\12Aug\\pe02_camp3_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\15Aug\\k38_diag1e_WGS.shp', '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\15Aug\\k38_diag1_WGS.shp', '//smb.isipd.d

In [47]:
def reproject_and_extract_coordinates(shapefile_path, output_crs):
    
    coordinates = []
    with fiona.open(shapefile_path) as shp:
        input_crs = shp.crs['init']  
        transformer = Transformer.from_crs(input_crs, output_crs, always_xy=True)

        for feature in shp:
            geom = shape(feature['geometry'])
            if geom.geom_type == 'Point':
                x, y = transformer.transform(geom.x, geom.y)
                print(feature['properties'].keys())
                point_id = feature['properties'].get('Point Id', 'Unknown')
                z = feature['properties'].get('WGS84 Elli', 0)  
                coordinates.append((point_id, x, y, z))
            elif geom.geom_type in ['LineString', 'Polygon']:
                for coord in geom.coords:
                    x, y = transformer.transform(coord[0], coord[1])
                    z = coord[2] if len(coord) > 2 else 0  
                    coordinates.append((x, y, z))

    return coordinates

In [49]:
import pandas as pd
import os

def save_coordinates_to_csv(shapefile_path, coordinates):
   
    # Generate the CSV filename based on the shapefile name
    base_name = os.path.splitext(os.path.basename(shapefile_path))[0]
    dir_name = os.path.dirname(shapefile_path)
    csv_file = os.path.join(dir_name, f"{base_name}_coordinates.csv")
    
    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(coordinates, columns=['ID', 'X', 'Y', 'Z (Ellipsoidal Height)'])
    df.to_csv(csv_file, index=False)
    
    print(f"Coordinates for {shapefile_path} saved to {csv_file}")


In [50]:
def save_coordinates_to_txt(shapefile_path, coordinates):
    
    # Generate the TXT filename based on the shapefile name
    base_name = os.path.splitext(os.path.basename(shapefile_path))[0]
    dir_name = os.path.dirname(shapefile_path)
    txt_file = os.path.join(dir_name, f"{base_name}_coordinates.txt")
    
    # Open the TXT file and write the data in a structured format
    with open(txt_file, 'w') as file:
        file.write("X\tY\tZ (Ellipsoidal Height)\n")  # Header for the TXT file
        
        for row in coordinates:
            x = row[1] if row[1] is not None else 0.0
            y = row[2] if row[2] is not None else 0.0
            z = row[3] if row[3] is not None else 0.0
            
            file.write(f"{x:.6f}\t{y:.6f}\t{z:.2f}\n")  # Write X, Y, Z
            
    print(f"Coordinates for {shapefile_path} saved to {txt_file}")



In [51]:
output_crs = 'EPSG:32608'  
shapefiles = find_shapefiles(main_directory)


for shapefile_path in shapefiles:
    print(f"Processing shapefile: {shapefile_path}")
    
    # Reproject and extract coordinates
    coordinates = reproject_and_extract_coordinates(shapefile_path, output_crs)
    
    # Save coordinates to a CSV file
    save_coordinates_to_csv(shapefile_path, coordinates)
    save_coordinates_to_txt(shapefile_path, coordinates)


Processing shapefile: //smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\10Aug\pe02_v1_WGS.shp
KeysView(<fiona.model.Properties object at 0x000002165F708A10>)
KeysView(<fiona.model.Properties object at 0x000002165F098D10>)
KeysView(<fiona.model.Properties object at 0x000002165F098D50>)
KeysView(<fiona.model.Properties object at 0x000002165F09AE10>)
KeysView(<fiona.model.Properties object at 0x000002165F09AC50>)
KeysView(<fiona.model.Properties object at 0x000002165F09B690>)
KeysView(<fiona.model.Properties object at 0x000002165F099A90>)
KeysView(<fiona.model.Properties object at 0x000002165F09B650>)
KeysView(<fiona.model.Properties object at 0x000002165F09A950>)
KeysView(<fiona.model.Properties object at 0x000002165F09B610>)
KeysView(<fiona.model.Properties object at 0x000002165F099E10>)
KeysView(<fiona.model.Properties object at 0x000002165F098E10>)
KeysView(<fiona.model.Properties object at 0x000002165F099510>)
KeysView(<fiona.model.Properties object at 0x0

PermissionError: [Errno 13] Permission denied: '//smb.isipd.dmawi.de/projects/sparc/personal_accounts/Mehriban/expedition2024\\11Aug\\pe02_h1_WGS_coordinates.csv'

function for calculating actual distances between the electrodes

In [53]:


# Function to calculate Euclidean distance between two points
def calculate_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

# Calculate distance from the first point to every other point
def calculate_distances_from_first(coordinates):
    first_point = coordinates[0]
    distances_from_first = [calculate_distance(first_point, coord) for coord in coordinates]
    return distances_from_first



In [69]:
coordinates = np.loadtxt(main_directory + '/16Aug/k38_old_125_coords.txt', usecols=(1, 2)) # change whenever need
distances = calculate_distances_from_first(coordinates)
distances

[0.0,
 1.2579769469949604,
 2.4849438621416686,
 3.723302297168288,
 4.959766627360385,
 6.201070955409846,
 7.466589716894601,
 8.680090437254984,
 9.95715551725504,
 11.196741132802757,
 12.426993723017018,
 13.65011080521789,
 14.923768726349946,
 16.136232552291176,
 17.36253221700366,
 18.64359056021493,
 19.881497856147515,
 21.04641881621235,
 22.279558029202967,
 23.48168205160177,
 24.665414044853023,
 25.889942139601345,
 27.13566238355372,
 28.38582903497328,
 29.623371870120433,
 30.87641405635803,
 32.11351746487696,
 33.36754328625883,
 34.62273277759448,
 35.86638438656255,
 37.10032875838406,
 38.34237720859868,
 39.6030255658054,
 40.84729612814062,
 42.07510352859309,
 43.33199476075951,
 44.581076612719826,
 45.8308861902011,
 47.07052891085061,
 48.32863275469832,
 49.56789369143274,
 50.82312289629483,
 52.04440481918764,
 53.29023701404755,
 54.541776171732494,
 55.78838339581002,
 57.04248460573397,
 58.26359220767709]